# Script to calibrate the position data of PBIG053
### Purpose:
Contact christopher.arrell@psi.ch

In [1]:
import epics as ep
import numpy as np
import matplotlib.pyplot as plt
from time import sleep
from bsread import source
from IPython.display import clear_output, display
from scipy.optimize import curve_fit
%matplotlib inline

## Setup and functions setup

In [3]:
# channels = ['SARFE10-LSCP1-FNS:CH0:VAL_GET','SARFE10-LSCP1-FNS:CH1:VAL_GET','SARFE10-LSCP1-FNS:CH2:VAL_GET','SARFE10-LSCP1-FNS:CH3:VAL_GET']
channels = ['SARFE10-LSCP1-FNS:CH0:VAL_GET','SARFE10-LSCP1-FNS:CH1:VAL_GET','SARFE10-LSCP1-FNS:CH2:VAL_GET','SARFE10-LSCP1-FNS:CH3:VAL_GET']

channel_names = ['Down','Up','Right','Left']
# PBPS_x_PV_name = 'SARFE10-PBPG050:MOTOR_X.VAL'
# PBPS_y_PV_name = 'SARFE10-PBPG050:MOTOR_Y.VAL'

# PBPS_x_PV = ep.PV(PBPS_x_PV_name)
# PBPS_y_PV = ep.PV(PBPS_y_PV_name)


def set_PBPS_x(pos):
    PBPS_x_PV.put(pos, wait=True)
def set_PBPS_y(pos):
    PBPS_y_PV.put(pos, wait=True)
    
def make_arrays(pvs, n_pulses):
    arrays = []
    for pv in pvs:
        val = pv.value

        dtype = get_dtype(val)
        shape = get_shape(val)
        shape = (n_pulses,) + shape

        arr = np.empty(shape, dtype)
        arrays.append(arr)

    return arrays

def PBPS_get_data(channels, n_pulses=100, wait_time=0.5):
    pvs = [ep.PV(ch) for ch in channels]

    n_channels = len(channels)
    counters = np.zeros(n_channels, dtype=int)

    arrays = make_arrays(pvs, n_pulses)


    def on_value_change(pv=None, ichannel=None, value=None, **kwargs):
        ivalue = counters[ichannel]
        arrays[ichannel][ivalue] = value

        counters[ichannel] += 1

        if counters[ichannel] == n_pulses:
            pv.clear_callbacks()


    for i, pv in enumerate(pvs):
        pv.add_callback(callback=on_value_change, pv=pv, ichannel=i)

    while not np.all(counters == n_pulses):
        sleep(wait_time)

    return(arrays)
def get_dtype(v):
    if is_array(v):
        return v.dtype
    else:
        return type(v)

def get_shape(v):
    if is_array(v):
        return v.shape
    else:
        return tuple()

def is_array(v):
    return isinstance(v, np.ndarray)

def PBPS_x_scan(Scan_x_range,channels, numShots):
        
    scan_mean = []
    scan_std = []
    scan_all = []
            
    for pos in Scan_x_range:
        set_PBPS_x(pos)  
        data = PBPS_get_data(channels, numShots)
        scan_mean.append([i.mean() for i in data])
        scan_std.append([i.std() for i in data])
        scan_all.append(data)
    return(np.asarray(scan_mean), np.asarray(scan_std), np.asarray(scan_all))

def PBPS_y_scan(Scan_y_range,channels, numShots):
        
    scan_mean = []
    scan_std = []
    scan_all = []
            
    for pos in Scan_y_range:
        set_PBPS_y(pos)  
        data = PBPS_get_data(channels, numShots)
        scan_mean.append([i.mean() for i in data])
        scan_std.append([i.std() for i in data])
        scan_all.append(data)
    return(np.asarray(scan_mean), np.asarray(scan_std), np.asarray(scan_all))

def PBPS_I_calibrate(channels, numShots):
        
    scan_mean = []
    scan_std = []
    scan_all = []

    data = PBPS_get_data(channels, numShots)
    scan_mean.append([i.mean() for i in data])
    scan_std.append([i.std() for i in data])
    scan_all.append(data)
    return(np.asarray(scan_mean), np.asarray(scan_std), np.asarray(scan_all))

def lin_fit(x,m,a):
    return m*x+a
def fit(xdata,ydata):
    popt, pcov = curve_fit(lin_fit, xdata, ydata)
    return(popt)

## User inputs

In [5]:
numShots = 500
# Scan_x_range = np.linspace(-1.0,1.5,10)
# Scan_y_range = np.linspace(-0.5,0.5,10)

## Scan and measure

In [6]:
scan_I_mean,scan_I_std, _ = PBPS_I_calibrate(channels, numShots*5)
norm_diodes = np.asarray([1 / tm / 4 for tm in scan_I_mean])

In [8]:
norm_diodes

array([[-0.000203  , -0.00029889, -0.00023488, -0.00029716]])

In [ ]:
scan_x_mean,scan_x_std, scan_x_data = PBPS_x_scan(Scan_x_range, channels, numShots)
#scan_x_mean,scan_x_std, scan_x_data = PBPS_x_scan(Scan_x_range, channels, numShots)

set_PBPS_x(-0.5)

In [ ]:
scan_y_mean,scan_y_std, _ = PBPS_y_scan(Scan_y_range, channels, numShots)
set_PBPS_y(0)

In [ ]:
popt_x = []
for i in range(0,scan_x_mean.shape[1]):
    popt_x.append(fit(Scan_x_range,scan_x_mean[:,i]))
popt_x =np.asarray(popt_x)

In [ ]:
plt.figure(figsize=[5,5])
for i in range(0,scan_x_mean.shape[1]):
    plt.errorbar(Scan_x_range, scan_x_mean[:,i], yerr=scan_x_std[:,i],label = channel_names[i])
    plt.plot(Scan_x_range,lin_fit(Scan_x_range,*popt_x[i,:]))
plt.legend()
plt.grid(True)
plt.title('PBIG calibration')
plt.xlabel('X position [mm]')
plt.ylabel('Inensity [arb]')

In [ ]:
plt.figure(figsize=[5,5])
for i in range(0,scan_y_mean.shape[1]):
    plt.errorbar(Scan_y_range, scan_y_mean[:,i], yerr=scan_y_std[:,i],label = channel_names[i])
plt.legend()
plt.grid(True)
plt.title('PBIG calibration')
plt.xlabel('Y position [mm]')
plt.ylabel('Inensity [arb]')

## Calibrate

In [ ]:
tmp = (scan_y_mean[:,0]-scan_y_mean[:,1])/(scan_y_mean[:,0]+scan_y_mean[:,1])
tmp_2 = (scan_x_mean[:,2]-scan_x_mean[:,3])/(scan_x_mean[:,2]+scan_x_mean[:,3])

In [ ]:
plt.figure()
plt.plot(Scan_y_range, tmp)

In [ ]:
plt.figure()
plt.plot(Scan_x_range, tmp_2)

In [ ]:
scan_y_mean

In [ ]:
norm_diodes[0,0]

In [ ]:
norm_diodes = np.asarray([1 / tm / 4 for tm in scan_I_mean])
scan_x_norm = (scan_x_mean[:,3]*norm_diodes[0,3]-scan_x_mean[:,2]*norm_diodes[0,2])/(scan_x_mean[:,3]*norm_diodes[0,3]+scan_x_mean[:,2]*norm_diodes[0,2])
scan_y_norm = (scan_y_mean[:,1]*norm_diodes[0,1]-scan_y_mean[:,0]*norm_diodes[0,0])/(scan_y_mean[:,1]*norm_diodes[0,1]+scan_y_mean[:,0]*norm_diodes[0,0])


In [ ]:
popt_norm_x = fit(Scan_x_range,scan_x_norm)
plt.figure()
plt.plot(Scan_x_range, scan_x_norm)
plt.plot(Scan_x_range,lin_fit(Scan_x_range,*popt_norm_x))

In [ ]:
popt_norm_y = fit(Scan_y_range,scan_y_norm)
plt.figure()
plt.plot(Scan_y_range, scan_y_norm)
plt.plot(Scan_y_range,lin_fit(Scan_y_range,*popt_norm_y))

In [10]:
#set intensities
ep.PV('SARES12-CVME-EVR0:CALCI.INPE').put(bytes(str(norm_diodes[0,0]), "utf8"))
ep.PV('SARES12-CVME-EVR0:CALCI.INPF').put(bytes(str(norm_diodes[0,1]), "utf8"))
ep.PV('SARES12-CVME-EVR0:CALCI.INPG').put(bytes(str(norm_diodes[0,2]), "utf8"))
ep.PV('SARES12-CVME-EVR0:CALCI.INPH').put(bytes(str(norm_diodes[0,3]), "utf8"))

1

In [ ]:
# set x position
ep.PV('SARFE10-PBIG050-EVR0:CALCX.INPE').put(bytes(str(norm_diodes[0,2]), "utf8"))
ep.PV('SARFE10-PBIG050-EVR0:CALCX.INPF').put(bytes(str(norm_diodes[0,3]), "utf8"))
ep.PV('SARFE10-PBIG050-EVR0:CALCX.INPI').put(bytes(str(popt_norm_x[0]*225), "utf8"))

## Test scans